# Panorama

Aplikacija ucitava dve JPG slike, korisnik, klikom misa, bira 6 tacaka na prvoj slici i istih tih 6 tacaka na drugoj slici, zatim se na taj nacin dobija matrica projektivnog preslikavanja kojom mozemo preslikati prvu sliku na drugu sliku. Na kraju spajamo modifikovanu prvu sliku sa drugom po horizontali i dobijamo panoramu.

In [1]:
import cv2
import numpy as np
from numpy import linalg as LA

In [2]:
def homogenizacija(tacka):
    return [tacka[0], tacka[1], 1]

In [3]:
def dehomogenizacija(tacka):
    return [tacka[0]/tacka[2], tacka[1]/tacka[2]]

In [4]:
def matricaKorespodencije(o, s):
    M = np.array([[0, 0, 0, -s[2]*o[0], -s[2]*o[1], -s[2]*o[2], s[1]*o[0], s[1]*o[1], s[1]*o[2]],
                 [s[2]*o[0], s[2]*o[1], s[2]*o[2], 0, 0, 0, -s[0]*o[0], -s[0]*o[1], -s[0]*o[2]]])
    return M

In [5]:
def dltAlgoritam(n, originali, slike):
    
    M1 = matricaKorespodencije(originali[0], slike[0])
    
    for i in range(1,n):
        M2 = matricaKorespodencije(originali[i], slike[i])
        M1 = np.vstack((M1, M2))
            
    U, D, V_transponovano = LA.svd(M1, full_matrices=True)
    
    P = V_transponovano[-1]
    P = P.reshape(3,3)
    return P

In [6]:
def normalizacija(n,tacke):
    afine = []

    tx = 0
    ty = 0
    for i in range(n):
        afine.append(dehomogenizacija(tacke[i]))
        tx = tx + afine[i][0]
        ty = ty + afine[i][1]
    tx = tx/n
    ty = ty/n

    G = np.array([[1,0,-tx], [0,1,-ty], [0,0,1]])

    for i in range(n):
        afine[i] = G.dot(homogenizacija(afine[i]))
   
    dist = 0
    for i in range(n):
        dist = dist + np.sqrt(np.square(afine[i][0]) + np.square(afine[i][1]))

    dist = dist/n

    S = np.array([[np.sqrt(2)/dist,0, 0], [0,np.sqrt(2)/dist,0], [0,0,1]])

    for i in range(n):
        afine[i] = S.dot(homogenizacija(afine[i]))

    T = S.dot(G)
  
    return T, afine

In [7]:
def normDltAlgoritam(n, originali, slike):
    originali_t, originali_n = normalizacija(n, originali)

    slike_t, slike_n = normalizacija(n, slike)

    MP = dltAlgoritam(n, originali_n, slike_n)

    M = MP.dot(originali_t)
    M = LA.inv(slike_t).dot(M)

    return np.round(M, decimals=5)

In [8]:
def click_event(event, x, y, flags, params):
    global i
    global naziv
    if event == cv2.EVENT_LBUTTONDOWN:
        tacke.append((x,y))
        font = cv2.FONT_HERSHEY_DUPLEX
        print("Tacka {} : {}, {}".format(i, x, y))
        i = i + 1
        cv2.putText(img, '(' + str(x) + ',' + str(y) + ')', (x,y), font, 0.9, (50, 50, 100), 2)
        cv2.imshow(naziv, img)

In [ ]:
global i
i = 1
tacke = []
naziv = 'Slika 1'

print('Slika 1')
img = cv2.imread("panorama1.jpg", 1)
img = cv2.resize(img,(800,600))
cv2.imshow(naziv, img)
cv2.setMouseCallback(naziv, click_event)

while True:
    if cv2.waitKey(1) & len(tacke) == 6:
        break
        
originali = []
for i in range(6):
    originali.append(homogenizacija(tacke[i]))

tacke = []
    
naziv = 'Slika 2'

i = 1

img2 = cv2.imread("panorama2.jpg", 1)
img2 = cv2.resize(img2,(800,600))

print('\nSlika 2')
img = cv2.imread("panorama2.jpg", 1)
img = cv2.resize(img,(800,600))

cv2.imshow(naziv, img)
cv2.setMouseCallback(naziv, click_event)

while True:
    if cv2.waitKey(1) & len(tacke) == 6:
        break
        
slike = []
for i in range(6):
    slike.append(homogenizacija(tacke[i]))    

for i in range(6):
    slike[i][0] = slike[i][0] + 800
    
P = normDltAlgoritam(6, originali, slike)
P = np.round(P,decimals=5)

img = cv2.imread("panorama1.jpg", 1)
img = cv2.resize(img,(800,600))

P = np.float64(P)
dst = cv2.warpPerspective(img,P,(800,600))
img1 = dst
# cv2.imshow('Nova slika 1',dst)

# M = np.float32([[1, 0, 100], [0, 1, 50]])
# translated = cv2.warpAffine(dst, M, (1600, 600))
# cv2.imshow('nova slika', translated)
# Na ovaj nacin mozemo da spojimo dve slike po horizontali,
# ali to nam ovde nije potrebno.

im_v = cv2.hconcat([img1, img2])
cv2.imshow('Panorama', im_v)

cv2.waitKey(0)

Slika 1
Tacka 1 : 778, 531
Tacka 2 : 770, 208
Tacka 3 : 662, 529
Tacka 4 : 660, 373
Tacka 5 : 657, 229
Tacka 6 : 657, 83

Slika 2
Tacka 1 : 514, 530
Tacka 2 : 507, 209
Tacka 3 : 389, 529
Tacka 4 : 385, 374
Tacka 5 : 384, 230
Tacka 6 : 381, 82
